In [1]:
import argparse
import os
import time
import sys

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import DataLoader
import torch.optim as optim

os.environ['CUDA_VISIBLE_DEVICES']="1,2"

In [2]:
from models import models
from dataset import openimages
from utils.loss import HardNegativeContrastiveLoss

In [4]:
class AverageMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [5]:
def train(train_loader, model, criterion, optimizer, epoch, print_freq=1000):
    #amp_handle = amp.init()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()

    model = model.train()
    print("Start training")
    end = time.time()
    for i, (imgs, caps) in enumerate(train_loader):
        if i%2 == 1:
                print("%2.2f"% (i/len(train_loader)*100), '\%', end='\r')
        input_imgs, target = imgs.cuda(), caps.cuda()
        

        data_time.update(time.time() - end)

        optimizer.zero_grad()
        
        output_imgs = model(input_imgs)
        
        
        loss = criterion(output_imgs, target)
        
        #with amp_handle.scale_loss(loss, optimizer) as scaled_loss:
        #    scaled_loss.backward()
        loss.backward()
        optimizer.step()
        
        losses.update(loss.item(), imgs.size(0))

        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0 or i == (len(train_loader) - 1):
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses))

    return losses.avg, batch_time.avg, data_time.avg

In [6]:
def validate(val_loader, model, criterion, print_freq=1000):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()

    model = model.eval()

    imgs_enc = list()
    caps_enc = list()
    end = time.time()
    for i, (imgs, caps, lengths) in enumerate(val_loader):

        input_imgs, input_caps = imgs.cuda(), caps.cuda()

        # measure data loading time
        data_time.update(time.time() - end)

        with torch.no_grad():
            output_imgs = model(input_imgs)
            loss = criterion(output_imgs, input_caps)

        imgs_enc.append(output_imgs.cpu().data.numpy())
        caps_enc.append(output_caps.cpu().data.numpy())
        losses.update(loss.item(), imgs.size(0))

        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0 or i == (len(val_loader) - 1):
            print('Data: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                      i, len(val_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses))

    recall  = eval_recall(imgs_enc, caps_enc)
    print(recall)
    return losses.avg, batch_time.avg, data_time.avg, recall

In [7]:
normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

prepro = transforms.Compose([
    transforms.RandomResizedCrop(224),

    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,
])

prepro_val = transforms.Compose([
    transforms.Resize((350, 350)),
    transforms.ToTensor(),
    normalize,
])

In [8]:
m = nn.DataParallel(models.ImageProjection().train().cuda())

In [9]:
for params in m.parameters():
    params.requires_grad=False

In [10]:
for params in m.module.projection.parameters():
    params.requires_grad=True

In [11]:
def collate_embeds(data):
    images, targets = zip(*data)
    images = torch.stack(images, 0)
    targets = torch.Tensor(np.stack(targets, 0))

    return images, targets

In [12]:
train_dataset = openimages.OpenImages(image_dir="/data/datasets/openimages/images/train/", 
                          image_file="/data/datasets/openimages/train-humanlabels.csv",
                          classes="/data/datasets/openimages/class-descriptions.csv", 
                          transform=prepro)

In [13]:
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, drop_last=True,
                            num_workers=20, collate_fn=collate_embeds, pin_memory=True)

In [14]:
opti = optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.001)

In [15]:
criterion = HardNegativeContrastiveLoss().cuda()

In [16]:
train(train_loader, m, criterion, opti, 0, print_freq=500)

Start training
Epoch: [0][0/26249]	Time 22.627 (22.627)	Data 11.988 (11.988)	Loss 4256.6763 (4256.6763)	
Epoch: [0][500/26249]	Time 0.655 (0.726)	Data 0.029 (0.057)	Loss 9688.1055 (10135.2469)	
Epoch: [0][1000/26249]	Time 0.705 (0.702)	Data 0.029 (0.044)	Loss 8685.4023 (9622.6638)	
Epoch: [0][1500/26249]	Time 0.650 (0.695)	Data 0.029 (0.040)	Loss 13249.8008 (9134.2250)	
Epoch: [0][2000/26249]	Time 0.652 (0.690)	Data 0.029 (0.038)	Loss 8582.8008 (9233.5652)	
Epoch: [0][2500/26249]	Time 0.672 (0.690)	Data 0.029 (0.036)	Loss 7336.8608 (9106.4438)	
Epoch: [0][3000/26249]	Time 0.653 (0.688)	Data 0.029 (0.036)	Loss 7079.2373 (9107.2860)	


/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3263561744 bytes but only got 0. Skipping tag 8
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2284257280 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2284978176 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19600506880 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py

/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 131584 bytes but only got 0. Skipping tag 512
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 655370 bytes but only got 0. Skipping tag 11
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1174935560 bytes but only got 0. Skipping tag 1032
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1376288 bytes but only got 0. Skipping tag 1032
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:76

/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1310780 bytes but only got 0. Skipping tag 4
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 393222 bytes but only got 0. Skipping tag 6
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 458759 bytes but only got 0. Skipping tag 17
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 524306 bytes but only got 0. Skipping tag 7
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarni

/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3680256 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 128 bytes but only got 0. Skipping tag 4
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 64 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 80 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly

...

/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 524296 bytes but only got 0. Skipping tag 6
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1310732 bytes but only got 65522. Skipping tag 9
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4194392 bytes but only got 0. Skipping tag 10
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 786468 bytes but only got 0. Skipping tag 8
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: User

/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 458758 bytes but only got 0. Skipping tag 6
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6815848 bytes but only got 0. Skipping tag 11
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5242960 bytes but only got 0. Skipping tag 14
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 393222 bytes but only got 65527. Skipping tag 3
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: Use

/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 131077 bytes but only got 0. Skipping tag 8
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 10485760 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3363429034 bytes but only got 0. Skipping tag 1
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:780: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 9. 
  warnings.warn(str(msg))


Epoch: [0][13000/26249]	Time 0.651 (0.680)	Data 0.029 (0.032)	Loss 12932.6338 (8675.9321)	
Epoch: [0][13500/26249]	Time 0.656 (0.680)	Data 0.029 (0.032)	Loss 9265.8535 (8693.5132)	
Epoch: [0][14000/26249]	Time 0.648 (0.679)	Data 0.029 (0.032)	Loss 7799.3457 (8668.8179)	
Epoch: [0][14500/26249]	Time 0.650 (0.679)	Data 0.029 (0.032)	Loss 4740.2051 (8662.9357)	
Epoch: [0][15000/26249]	Time 0.653 (0.678)	Data 0.029 (0.032)	Loss 7933.9785 (8666.2452)	
Epoch: [0][15500/26249]	Time 0.652 (0.678)	Data 0.029 (0.032)	Loss 10573.2061 (8677.0085)	
Epoch: [0][16000/26249]	Time 0.652 (0.678)	Data 0.029 (0.031)	Loss 8105.0508 (8636.2572)	
Epoch: [0][16500/26249]	Time 0.650 (0.677)	Data 0.030 (0.031)	Loss 7460.2148 (8620.0805)	
Epoch: [0][17000/26249]	Time 0.654 (0.677)	Data 0.029 (0.031)	Loss 8553.9922 (8622.4562)	
Epoch: [0][17500/26249]	Time 0.653 (0.677)	Data 0.029 (0.031)	Loss 6799.8330 (8606.2590)	
Epoch: [0][18000/26249]	Time 0.650 (0.677)	Data 0.029 (0.031)	Loss 9029.6943 (8613.1153)	
Epoch: [

(8561.166979984655, 0.6741539630405226, 0.030716235853203402)

In [ ]:
for param in m.parameters():
    param.requires_grad = True

# Keep the first layer of resnet frozen
for i in range(0, 6):
    for param in m.module.base_layer[i].parameters():
        param.requires_grad = False

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=450, shuffle=True, drop_last=True,
                            num_workers=20, collate_fn=collate_embeds, pin_memory=True)
opti = optim.Adam(filter(lambda p: p.requires_grad, m.module.base_layer.parameters()), lr=0.00025)

In [ ]:
for i in range(1,5):
    train(train_loader, m, criterion, opti, i, print_freq=500)